In [1]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path
import tensorflow as tf

In [3]:
df = pd.read_pickle('final_processed_invoker_data')

In [12]:
columnDict = {}
for i, v in enumerate(df.columns.values):
    columnDict[i] = v
indexDict = {v: k for k, v in columnDict.items()}
numberEntries = len(df)
batchSize = int(numberEntries/20)

In [ ]:
matchItemsTensor = tf.placeholder(tf.float32, shape=shape=[None, None, 266])
batched_data = tf.train.batch(
    tensors=[matchItemsTensor],
    batch_size=batchSize,
    dynamic_pad=True,
    name="lstm_batch"
)
lstmCell = tf.contrib.rnn.BasicLSTMCell(50)